In [1]:
import sys, os, pygmt
sys.path.insert(0, '/home/581/da1339/AFIM/src/python')
from fast_ice_processor import FastIceProcessor
#from fast_ice_plotter   import FastIcePlotter
from pack_ice_processor import PackIceProcessor
from sea_ice_plotter    import SeaIcePlotter
#from grounded_iceberg_processor import GroundedIcebergProcessor
import numpy            as np
import pandas           as pd
import xarray           as xr
from datetime           import timedelta, date, datetime

In [3]:
dt_test  = datetime(1993,8,14)
sim_name  = 'Cstar-max'
PI_proc  = PackIceProcessor(sim_name)
PI       = PI_proc.process_window(dt_test, save_zarr=False)

2025-03-30 06:32:41,527 - INFO - 

Processing PI window centered on 1993-08-14 00:00:00 for southern hemisphere
2025-03-30 06:32:57,266 - INFO - Loaded dataset with shape: Frozen({'time': 15, 'nj': 1080, 'ni': 1440})
INFO:Cstar-max_PI:Loaded dataset with shape: Frozen({'time': 15, 'nj': 1080, 'ni': 1440})
2025-03-30 06:32:57,821 - INFO - Regridding uvel and vvel to T-grid...
INFO:Cstar-max_PI:Regridding uvel and vvel to T-grid...
--------------------------------------------------------------------------

  Local host:   gadi-hmem-clx-0039
  Local device: mlx5_0
--------------------------------------------------------------------------
2025-03-30 06:37:39,690 - INFO - compute pack ice 1D variables:
INFO:Cstar-max_PI:compute pack ice 1D variables:
2025-03-30 06:37:41,078 - INFO - 	time taken: 1.3746061325073242 seconds
INFO:Cstar-max_PI:	time taken: 1.3746061325073242 seconds
2025-03-30 06:37:41,081 - INFO - 	time taken: 1.3779168128967285 seconds
INFO:Cstar-max_PI:	time taken: 1.3779168

In [ ]:
dt0_str  = "1993-01-01"
dtN_str  = "1993-12-31"
sim_names = [
    "ustar-low", "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min",
    "ktens-nil", "ktens-min", "ktens-max", "gi-max", "gi-min",
    "Rothrock", "elps-pack", "elps-fast", "elps-max", "elps-def",
    "elps-mid", "baseline"
]
for sim_name in sim_names:
    FI_plot  = FastIcePlotter(sim_name, dt0_str, dtN_str, show_figs=False, overwrite=True, single_figure=False)
    FI_plot.prepare_data_for_plotting(var_name='FISH_SD')
    FI_plot.plot_regions()

In [2]:
dt0_str  = "1993-01-30"
dtN_str  = "1994-12-31"
sim_name = 'ktens-max'
plotter = SeaIcePlotter(sim_name, dt0_str, dtN_str, show_figs=True, ice_type='PI')
plotter.plot_timeseries(var_names=["PI_STRESS"])
plotter.plot_timeseries_compare(
    comparison_name = "tensile-stars",
    sim_names       = ["baseline", "ktens-max", "Pstar-max", "Cstar-min"],
    var_name        = "PIA",
    label_dict      = {"baseline"  : "tensile = 0.2, Pstar = 2.75e4, Cstar = 20",
                       "ktens-max" : "tensile = 0.3, Pstar = 2.75e4, Cstar = 20",
                       "Pstar-max" : "tensile = 0.2, Pstar = 5e4   , Cstar = 20",
                       "Cstar-min" : "tensile = 0.2, Pstar = 2.75e4, Cstar = 10"})

FileNotFoundError: No Zarr files found in /g/data/gv90/da1339/afim_output/ktens-max/PI

In [ ]:
afim_anal = AFIM.analysis()
if 'hs' in list(afim_anal.plot_var_dict.keys()):
    print(False)

In [ ]:
# After the loop, create an animation using ffmpeg (example command)
# Assuming you're working in a Unix-like environment with ffmpeg installed:
for region_name in regions:
    D_save = f"/g/data/jk72/da1339/GRAPHICAL/AFIM/{run_name}/{var_name}/regional_cartopy/{region_name}"
    D_ani  = f"/g/data/jk72/da1339/GRAPHICAL/animations"
    os.system(f"ffmpeg -framerate 2 -pattern_type glob -i '{D_save}/*.png' -c:v libx264 -pix_fmt yuv420p {D_ani}/{run_name}_{var_name}_{region_name}.mp4")

In [ ]:
from pathlib import Path

sim_names = [
    "Cstar-max", "Cstar-min", "Pstar-max", "Pstar-min", "Rothrock", "baseline",
    "elps-def", "elps-fast", "elps-max", "elps-mid", "elps-pack", "gi-max",
    "gi-min", "ktens-max", "ktens-min", "ktens-nil", "oras-sfc-ctl", "ustar-low", "van_achter"]

pbs_template = """#!/bin/bash
#PBS -N PI-proc-{name}
#PBS -P gv90
#PBS -l walltime=10:00:00
#PBS -q normalbw
#PBS -l mem=20GB
#PBS -l ncpus=28
#PBS -l storage=gdata/cj50+gdata/jk72+scratch/jk72+gdata/ik11+gdata/hh5+gdata/rt52+gdata/gb6+gdata/gv90
#PBS -M daniel.atwater@utas.edu.au

module purge
module use /g/data/hh5/public/modules
module load conda/analysis3

cd ~/AFIM/src/python/
python3 ./compute_pack_ice.py {name}
"""

output_dir = Path("/home/581/da1339/AFIM/src/sh/analysis")
output_dir.mkdir(exist_ok=True)

for sim in sim_names:
    script_content = pbs_template.format(name=sim)
    script_path = Path(output_dir,f"PI-proc_{sim}.pbs")
    with open(script_path, "w") as f:
        f.write(script_content)
print(f"✓ Generated {len(sim_names)} PBS scripts in: {output_dir.resolve()}")